# Kapitola 16: Uvod do pravdepodobnosti - Monty Hall a Bayesova veta

**Blok 2: Prohledavaci algoritmy a optimalizace**

---

## Co se naucite

V teto kapitole:
- Pochopite zakladni pojmy **pravdepodobnosti** a jejich vyznam pro AI
- Simulujete slavny **Monty Hall problem** a dokazete, proc intuice klame
- Naucite se **Bayesovu vetu** - zaklad pro strojove uceni
- Pochopite **podminenou pravdepodobnost** a jak nova informace meni nase presvedceni
- Vytvorite **vizualizace** experimentu s pravdepodobnosti

---

## Motivace: Proc AI potrebuje pravdepodobnost?

AI casto pracuje s **nejistotou**:
- Jaka je pravdepodobnost, ze tento email je spam?
- Jaka je pravdepodobnost, ze pacient ma nemoc X?
- Jaka je pravdepodobnost, ze uzivatel klikne na reklamu?

Pravdepodobnost nam umoznuje:
1. **Kvantifikovat nejistotu** - misto "mozna" rikame "75%"
2. **Aktualizovat presvedceni** na zaklade novych dukazu
3. **Delat optimalni rozhodnuti** i v nejistych situacich

In [ ]:
# Import knihoven
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import random

# Nastaveni pro reprodukovatelnost
np.random.seed(42)

print("Knihovny nacteny!")
print("Pripraveno pro experimenty s pravdepodobnosti.")

---

## Cast 1: Zakladni pojmy pravdepodobnosti

### Definice

| Pojem | Definice | Priklad |
|-------|----------|--------|
| **Jev** | Vysledek, ktery muze nastat | "Padne 6 na kostce" |
| **P(A)** | Pravdepodobnost jevu A (0 az 1) | P(padne 6) = 1/6 |
| **P(A \| B)** | Podminena pravdepodobnost A za podmínky B | P(auto za dvermi 2 \| moderator otevrel dvere 3) |

### Zakladni pravidla

1. **Pravidlo scitani** (pro NEBO): P(A nebo B) = P(A) + P(B) - P(A a B)
2. **Pravidlo nasobeni** (pro A ZAROVEN): P(A a B) = P(A) * P(B|A)

In [ ]:
# Priklad: Simulace hodu kostkou

def simuluj_hod_kostkou(pocet_hodu=10000):
    """
    Simuluje hod kostkou a spocita cetnosti.
    """
    vysledky = np.random.randint(1, 7, pocet_hodu)  # 1-6
    cetnosti = Counter(vysledky)
    
    return cetnosti

# Spustime simulaci
pocet = 10000
cetnosti = simuluj_hod_kostkou(pocet)

print(f"Simulace {pocet} hodu kostkou:")
print("="*40)

for cislo in range(1, 7):
    pocet_vyskytu = cetnosti[cislo]
    pravdepodobnost = pocet_vyskytu / pocet
    teoreticka = 1/6
    print(f"Cislo {cislo}: {pocet_vyskytu:5d} ({pravdepodobnost:.4f}) | Teoreticky: {teoreticka:.4f}")

# Vizualizace
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Graf 1: Cetnosti
cisla = list(range(1, 7))
hodnoty = [cetnosti[c] for c in cisla]
axes[0].bar(cisla, hodnoty, color='steelblue', edgecolor='black')
axes[0].axhline(y=pocet/6, color='red', linestyle='--', label='Ocekavana hodnota')
axes[0].set_xlabel('Cislo na kostce')
axes[0].set_ylabel('Pocet vyskytu')
axes[0].set_title(f'Cetnosti po {pocet} hodech')
axes[0].legend()

# Graf 2: Pravdepodobnosti
probs = [cetnosti[c] / pocet for c in cisla]
axes[1].bar(cisla, probs, color='green', edgecolor='black', alpha=0.7, label='Empiricka')
axes[1].axhline(y=1/6, color='red', linestyle='--', label='Teoreticka (1/6)')
axes[1].set_xlabel('Cislo na kostce')
axes[1].set_ylabel('Pravdepodobnost')
axes[1].set_title('Empiricka vs teoreticka pravdepodobnost')
axes[1].legend()
axes[1].set_ylim(0, 0.25)

plt.tight_layout()
plt.show()

---

## Cast 2: Monty Hall problem

### Pravidla souteze

1. Stojite pred **3 zavrenymy dvermi**
2. Za jednimi je **auto**, za zbylymi jsou **kozy**
3. Vyberete si jedny dvere (napr. c. 1)
4. Moderator, ktery vi co je kde, **otevre jine dvere s kozou** (napr. c. 3)
5. Nabidne vam: **"Chcete zmenit volbu na dvere c. 2?"**

### Intuitivni odpoved

Vetsina lidi si mysli: "Ted je to 50:50, takze na tom nezalezi."

### Spravna odpoved

**Vzdy se vyplati zmenit dvere!** Sance je 2/3 oproti 1/3.

Pojdme si to dokazat simulaci!

In [ ]:
def monty_hall_simulace(pocet_her=10000, strategie="zmenit"):
    """
    Simuluje Monty Hall problem.
    
    Args:
        pocet_her: Pocet simulovanych her
        strategie: "zmenit" nebo "zustat"
    
    Returns:
        Pocet vyher
    """
    vyhry = 0
    
    for _ in range(pocet_her):
        # 1. Nahodne umistime auto za jedny ze tri dveri (0, 1, 2)
        dvere_s_autem = np.random.randint(0, 3)
        
        # 2. Hrac si nahodne vybere jedny dvere
        hracova_volba = np.random.randint(0, 3)
        
        # 3. Moderator otevre jedny ze zbyvajicich dveri, kde je koza
        mozne_k_otevreni = [d for d in range(3) 
                           if d != dvere_s_autem and d != hracova_volba]
        otevrene_moderatorem = np.random.choice(mozne_k_otevreni)
        
        # 4. Urcime dvere pro zmenu (ty, ktere nejsou otevrene ani puvodni volba)
        dvere_pro_zmenu = [d for d in range(3) 
                          if d != hracova_volba and d != otevrene_moderatorem][0]
        
        # 5. Vyhodnotime vysledek podle strategie
        if strategie == "zmenit":
            finalni_volba = dvere_pro_zmenu
        else:  # zustat
            finalni_volba = hracova_volba
        
        if finalni_volba == dvere_s_autem:
            vyhry += 1
    
    return vyhry


# Spustime simulaci pro obe strategie
pocet_her = 10000

vyhry_zmenit = monty_hall_simulace(pocet_her, "zmenit")
vyhry_zustat = monty_hall_simulace(pocet_her, "zustat")

print("MONTY HALL SIMULACE")
print("="*50)
print(f"Pocet her: {pocet_her}")
print("")
print(f"Strategie 'ZUSTAT':  {vyhry_zustat:5d} vyher ({vyhry_zustat/pocet_her*100:.1f}%)")
print(f"Strategie 'ZMENIT':  {vyhry_zmenit:5d} vyher ({vyhry_zmenit/pocet_her*100:.1f}%)")
print("="*50)
print("")
print(f"Pomer: {vyhry_zmenit/vyhry_zustat:.2f}x lepsi sance pri zmene!")
print("")
print("Teoreticka pravdepodobnost:")
print(f"  ZUSTAT: 1/3 = 33.33%")
print(f"  ZMENIT: 2/3 = 66.67%")

In [ ]:
# Vizualizace konvergence k teoretickym hodnotam

def sleduj_konvergenci(max_her=5000):
    """
    Sleduje, jak se empiricka pravdepodobnost priblizuje teoreticke.
    """
    vyhry_zmenit = 0
    vyhry_zustat = 0
    
    historie_zmenit = []
    historie_zustat = []
    
    for i in range(1, max_her + 1):
        dvere_s_autem = np.random.randint(0, 3)
        hracova_volba = np.random.randint(0, 3)
        
        mozne_k_otevreni = [d for d in range(3) 
                           if d != dvere_s_autem and d != hracova_volba]
        otevrene_moderatorem = np.random.choice(mozne_k_otevreni)
        
        dvere_pro_zmenu = [d for d in range(3) 
                          if d != hracova_volba and d != otevrene_moderatorem][0]
        
        # Strategie ZMENIT
        if dvere_pro_zmenu == dvere_s_autem:
            vyhry_zmenit += 1
        
        # Strategie ZUSTAT
        if hracova_volba == dvere_s_autem:
            vyhry_zustat += 1
        
        historie_zmenit.append(vyhry_zmenit / i)
        historie_zustat.append(vyhry_zustat / i)
    
    return historie_zmenit, historie_zustat


# Spustime sledovani
max_her = 5000
hist_zmenit, hist_zustat = sleduj_konvergenci(max_her)

# Vizualizace
plt.figure(figsize=(12, 6))

x = range(1, max_her + 1)
plt.plot(x, hist_zmenit, label='Strategie ZMENIT', color='green', alpha=0.8)
plt.plot(x, hist_zustat, label='Strategie ZUSTAT', color='orange', alpha=0.8)

plt.axhline(y=2/3, color='green', linestyle='--', alpha=0.5, label='Teoreticka (2/3)')
plt.axhline(y=1/3, color='orange', linestyle='--', alpha=0.5, label='Teoreticka (1/3)')

plt.xlabel('Pocet her')
plt.ylabel('Pravdepodobnost vyhry')
plt.title('Monty Hall: Konvergence k teoretickym hodnotam')
plt.legend(loc='right')
plt.grid(True, alpha=0.3)
plt.ylim(0, 1)

plt.tight_layout()
plt.show()

---

## Cast 3: Proc to tak je? Podminena pravdepodobnost

### Vysvetleni

Na zacatku:
- P(auto za vasimi dvermi) = **1/3**
- P(auto za NEKTERYMI z ostatnich dveri) = **2/3**

Kdyz moderator otevre jedny dvere s kozou:
- Vase puvodni volba ma stale **1/3**
- Cela pravdepodobnost 2/3 se "prelije" na jedine zbyvajici dvere!

### Klic: Moderatorova akce NENI nahodna!

Moderator **vzdy** otevre dvere s kozou. Tym vam dava **novou informaci**.

In [ ]:
# Vizualizace rozhodnuti - strom pravdepodobnosti

def zobraz_strom_rozhodnuti():
    """
    Zobrazi strom rozhodnuti pro Monty Hall.
    """
    fig, ax = plt.subplots(figsize=(14, 8))
    
    # Skryjeme osy
    ax.axis('off')
    
    # Texty
    ax.text(0.5, 0.95, 'Monty Hall - Strom pravdepodobnosti', 
            ha='center', fontsize=16, fontweight='bold')
    
    # Uroven 1: Vase volba
    ax.text(0.5, 0.85, 'Vyberete dvere', ha='center', fontsize=12)
    
    # Uroven 2: Kde je auto
    ax.text(0.2, 0.7, 'Auto za vasimi dvermi\nP = 1/3', 
            ha='center', fontsize=10, bbox=dict(boxstyle='round', facecolor='lightcoral'))
    ax.text(0.8, 0.7, 'Auto za jinymi dvermi\nP = 2/3', 
            ha='center', fontsize=10, bbox=dict(boxstyle='round', facecolor='lightgreen'))
    
    # Sipky
    ax.annotate('', xy=(0.2, 0.72), xytext=(0.5, 0.82),
                arrowprops=dict(arrowstyle='->', color='black'))
    ax.annotate('', xy=(0.8, 0.72), xytext=(0.5, 0.82),
                arrowprops=dict(arrowstyle='->', color='black'))
    
    # Uroven 3: Moderator otevre
    ax.text(0.2, 0.5, 'Moderator otevre\njine dvere s kozou', 
            ha='center', fontsize=10)
    ax.text(0.8, 0.5, 'Moderator otevre\njine dvere s kozou', 
            ha='center', fontsize=10)
    
    # Sipky
    ax.annotate('', xy=(0.2, 0.52), xytext=(0.2, 0.65),
                arrowprops=dict(arrowstyle='->', color='black'))
    ax.annotate('', xy=(0.8, 0.52), xytext=(0.8, 0.65),
                arrowprops=dict(arrowstyle='->', color='black'))
    
    # Uroven 4: Vysledky
    ax.text(0.1, 0.3, 'ZUSTAT\n= VYHRA', 
            ha='center', fontsize=11, fontweight='bold',
            bbox=dict(boxstyle='round', facecolor='gold'))
    ax.text(0.3, 0.3, 'ZMENIT\n= PROHRA', 
            ha='center', fontsize=11,
            bbox=dict(boxstyle='round', facecolor='lightgray'))
    
    ax.text(0.7, 0.3, 'ZUSTAT\n= PROHRA', 
            ha='center', fontsize=11,
            bbox=dict(boxstyle='round', facecolor='lightgray'))
    ax.text(0.9, 0.3, 'ZMENIT\n= VYHRA', 
            ha='center', fontsize=11, fontweight='bold',
            bbox=dict(boxstyle='round', facecolor='gold'))
    
    # Sipky k vysledkum
    ax.annotate('', xy=(0.1, 0.35), xytext=(0.15, 0.47),
                arrowprops=dict(arrowstyle='->', color='black'))
    ax.annotate('', xy=(0.3, 0.35), xytext=(0.25, 0.47),
                arrowprops=dict(arrowstyle='->', color='black'))
    ax.annotate('', xy=(0.7, 0.35), xytext=(0.75, 0.47),
                arrowprops=dict(arrowstyle='->', color='black'))
    ax.annotate('', xy=(0.9, 0.35), xytext=(0.85, 0.47),
                arrowprops=dict(arrowstyle='->', color='black'))
    
    # Zaver
    ax.text(0.5, 0.1, 'ZUSTAT vyhraje v 1 z 3 pripadu (33%)\nZMENIT vyhraje ve 2 z 3 pripadu (67%)', 
            ha='center', fontsize=12, fontweight='bold',
            bbox=dict(boxstyle='round', facecolor='lightyellow', edgecolor='orange'))
    
    plt.tight_layout()
    plt.show()

zobraz_strom_rozhodnuti()

---

## Cast 4: Bayesova veta

Bayesova veta je zakladem pro aktualizaci presvedceni na zaklade novych dukazu.

### Vzorec

$$P(A|B) = \frac{P(B|A) \cdot P(A)}{P(B)}$$

### Vyznam

| Pojem | Nazev | Vyznam |
|-------|-------|--------|
| P(A) | Prior | Puvodni presvedceni pred novymi dukazy |
| P(B\|A) | Likelihood | Pravdepodobnost dukazu za predpokladu A |
| P(A\|B) | Posterior | Aktualizovane presvedceni po dukazu |
| P(B) | Evidence | Celkova pravdepodobnost dukazu |

### Priklad: Medicinska diagnostika

Test na nemoc ma:
- 99% senzitivitu (spravne zachyti nemocne)
- 95% specificitu (spravne oznaci zdrave)
- Nemoc ma 1% populace

**Otazka**: Pokud je test pozitivni, jaka je pravdepodobnost, ze mate nemoc?

In [ ]:
def bayesuv_test_nemoci(prevalence, senzitivita, specificita):
    """
    Vypocita pravdepodobnost nemoci pri pozitivnim testu.
    
    Args:
        prevalence: P(nemoc) - % populace s nemoci
        senzitivita: P(pozitivni | nemoc) - true positive rate
        specificita: P(negativni | zdravy) - true negative rate
    
    Returns:
        P(nemoc | pozitivni test)
    """
    # Prior
    p_nemoc = prevalence
    p_zdravy = 1 - prevalence
    
    # Likelihood
    p_pozitivni_nemocny = senzitivita
    p_pozitivni_zdravy = 1 - specificita  # False positive rate
    
    # Evidence: P(pozitivni test)
    p_pozitivni = (p_pozitivni_nemocny * p_nemoc + 
                   p_pozitivni_zdravy * p_zdravy)
    
    # Posterior: P(nemoc | pozitivni test)
    p_nemoc_pozitivni = (p_pozitivni_nemocny * p_nemoc) / p_pozitivni
    
    return p_nemoc_pozitivni, p_pozitivni


# Parametry testu
prevalence = 0.01      # 1% populace ma nemoc
senzitivita = 0.99     # 99% spravne zachyti nemocne
specificita = 0.95     # 95% spravne oznaci zdrave

p_nemoc, p_pozitivni = bayesuv_test_nemoci(prevalence, senzitivita, specificita)

print("BAYESOVA ANALYZA MEDICINSKHO TESTU")
print("="*50)
print(f"Prevalence nemoci:     {prevalence*100:.1f}%")
print(f"Senzitivita testu:     {senzitivita*100:.1f}%")
print(f"Specificita testu:     {specificita*100:.1f}%")
print("")
print(f"P(pozitivni test):     {p_pozitivni*100:.2f}%")
print("")
print(f"P(nemoc | poz. test):  {p_nemoc*100:.1f}%")
print("="*50)
print("")
print("PREKVAPENI: I pri pozitivnim testu mate jen ~17% sanci,")
print("ze skutecne mate nemoc! To proto, ze nemoc je vzacna.")

In [ ]:
# Vizualizace: Jak prevalence ovlivnuje vysledek

prevalence_range = np.linspace(0.001, 0.2, 100)
posteriors = []

for prev in prevalence_range:
    p_nemoc, _ = bayesuv_test_nemoci(prev, 0.99, 0.95)
    posteriors.append(p_nemoc)

plt.figure(figsize=(10, 6))
plt.plot(prevalence_range * 100, np.array(posteriors) * 100, 
         linewidth=2, color='blue')

plt.axhline(y=50, color='red', linestyle='--', alpha=0.5, label='50% sance')
plt.axvline(x=1, color='green', linestyle=':', alpha=0.5, label='1% prevalence')

plt.xlabel('Prevalence nemoci v populaci (%)')
plt.ylabel('P(nemoc | pozitivni test) (%)')
plt.title('Vliv prevalence na diagnostickou hodnotu testu\n(Senzitivita 99%, Specificita 95%)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Pri vzacnych nemocich i velmi presny test dava mnoho falesne pozitivnich!")

---

## Cast 5: Monty Hall se 10 dvermi

Co se stane, kdyz zvetsime pocet dveri?

In [ ]:
def monty_hall_n_dveri(n_dveri, pocet_her=10000):
    """
    Monty Hall s N dvermi.
    Moderator otevre N-2 dveri s kozami.
    """
    vyhry_zmenit = 0
    vyhry_zustat = 0
    
    for _ in range(pocet_her):
        dvere_s_autem = np.random.randint(0, n_dveri)
        hracova_volba = np.random.randint(0, n_dveri)
        
        # Moderator otevre vsechny ostatni dvere krome jedne (s autem nebo te, na kterou hrac zmeni)
        # Zbydou 2 dvere: hracova a jedna dalsi
        
        if hracova_volba == dvere_s_autem:
            # Hrac vybral auto - zmena = prohra
            vyhry_zustat += 1
        else:
            # Hrac nevybral auto - zmena = vyhra (moderator ukazal vsechny kozy krome jedne)
            vyhry_zmenit += 1
    
    return vyhry_zustat / pocet_her, vyhry_zmenit / pocet_her


# Testujeme ruzne pocty dveri
dvere_values = [3, 5, 10, 20, 50, 100]

print("MONTY HALL S RUZNYM POCTEM DVERI")
print("="*60)
print(f"{'Pocet dveri':<15} {'P(zustat)':<15} {'P(zmenit)':<15} {'Teoreticky'}")
print("-"*60)

vysledky = []
for n in dvere_values:
    p_zustat, p_zmenit = monty_hall_n_dveri(n, 10000)
    teoreticky_zmenit = (n - 1) / n
    vysledky.append((n, p_zustat, p_zmenit, teoreticky_zmenit))
    print(f"{n:<15} {p_zustat*100:<15.1f}% {p_zmenit*100:<15.1f}% {teoreticky_zmenit*100:.1f}%")

print("="*60)
print("\nS vice dvermi je vyhoda zmeny jeste vetsi!")

In [ ]:
# Vizualizace
dvere = [v[0] for v in vysledky]
p_zmenit = [v[2] for v in vysledky]
p_zustat = [v[1] for v in vysledky]

x = range(len(dvere))
width = 0.35

fig, ax = plt.subplots(figsize=(12, 6))

bars1 = ax.bar([i - width/2 for i in x], [p*100 for p in p_zustat], width, 
               label='Zustat', color='orange')
bars2 = ax.bar([i + width/2 for i in x], [p*100 for p in p_zmenit], width, 
               label='Zmenit', color='green')

ax.set_xlabel('Pocet dveri')
ax.set_ylabel('Pravdepodobnost vyhry (%)')
ax.set_title('Monty Hall: Vliv poctu dveri na strategii')
ax.set_xticks(x)
ax.set_xticklabels(dvere)
ax.legend()
ax.set_ylim(0, 105)

# Pridame hodnoty nad sloupce
for bar in bars1:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.0f}%', ha='center', va='bottom', fontsize=9)

for bar in bars2:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.0f}%', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

---

## Cviceni k procviceni

### Cviceni 1: Dva hody kostkou
Simulujte 10000 hodu dvema kostkami. Jaka je pravdepodobnost, ze soucet je 7?

### Cviceni 2: Birthday problem
Kolik lidi musi byt v mistnosti, aby pravdepodobnost, ze dva maji stejne narozeniny, byla > 50%?

### Cviceni 3: Bayesuv spam filtr
Pokud email obsahuje slovo "vyhra", P(spam|vyhra) = 0.8. Pokud je legitimni, P(vyhra|legitimni) = 0.01. Spocitejte.

### Cviceni 4: Opakujici se test
Co kdyz test na nemoc opakujeme 2x? Jak se zmeni P(nemoc|oba pozitivni)?

In [ ]:
# Prostor pro vase reseni - Cviceni 1

def simuluj_soucet_kostek(pocet_hodu=10000):
    """
    Simuluje hod dvema kostkami a pocita pravdepodobnost souctu 7.
    """
    # VAS KOD ZDE:
    # kostka1 = np.random.randint(1, 7, pocet_hodu)
    # kostka2 = np.random.randint(1, 7, pocet_hodu)
    # soucty = kostka1 + kostka2
    # pocet_sedmicek = np.sum(soucty == 7)
    # ...
    pass

print("Cviceni 1: Doplnte kod vyse")
print("Teoreticka pravdepodobnost souctu 7: 6/36 = 16.67%")

---

## Shrnuti kapitoly

### Co jsme se naucili:

1. **Zakladni pravdepodobnost** - jevy, P(A), pravidla scitani a nasobeni
2. **Monty Hall problem** - jak simulace dokazuje proti-intuitivni vysledek
3. **Podminena pravdepodobnost** - jak nova informace meni nase presvedceni
4. **Bayesova veta** - zaklad pro aktualizaci presvedceni v AI
5. **Medicinska diagnostika** - proc i dobry test muze byt zaváějící

### Klicove vzorce:

| Vzorec | Nazev | Pouziti |
|--------|-------|--------|
| P(A nebo B) = P(A) + P(B) | Scitani | Jevy se vylucuji |
| P(A a B) = P(A) * P(B\|A) | Nasobeni | Zavisle jevy |
| P(A\|B) = P(B\|A)*P(A)/P(B) | Bayes | Aktualizace presvedceni |

### Dalsi kroky:

V dalsi kapitole se podivame na **Naivni Bayesuv klasifikator** - jak pouzit
tyto principy k vytvoreni skutecneho AI modelu pro klasifikaci textu!

---

*Kapitola 16 - Uvod do pravdepodobnosti*